In [ ]:
![ -d QAOAKit ] || git clone https://github.com/melody0123/QAOAKit.git

import os
os.chdir("QAOAKit")
print(os.getcwd()) # debugging output

!pip install .

os.chdir("..")
print(os.getcwd()) # debugging output

In [ ]:
!pip install pyzx
!pip install 'trasyn[qiskit,cupy-cuda12]'
!pip install matplotlib pylatexenc

In [ ]:
!python -m QAOAKit.build_tables

In [7]:
import networkx as nx
from qiskit import qpy
from qiskit_aer import AerSimulator
from QAOAKit import opt_angles_for_graph, angles_to_qaoa_format
from QAOAKit.qaoa import get_maxcut_qaoa_circuit

compute_on_the_fly = False
global qc
global new_qc
if compute_on_the_fly:
    # build a random 3-regular graph with 10 nodes
    graph = nx.random_regular_graph(d=3, n=10, seed=314)

    # grab optimal angles
    p = 3
    angles = angles_to_qaoa_format(opt_angles_for_graph(graph,p))

    # build circuit
    qc = get_maxcut_qaoa_circuit(graph, angles['beta'], angles['gamma'])

    # remove save_state operation
    new_qc = qc.copy_empty_like()
    for inst in qc:
        op = inst.operation
        if op.name != 'save_state':
            new_qc.append(op, inst.qubits, inst.clbits)
    
    # save circuit
    with open('algo_level_qc.qpy', 'wb') as qc_file:
        qpy.dump(qc, qc_file)
    with open('algo_level_qc_no_save_state.qpy', 'wb') as qc_file:
        qpy.dump(new_qc, qc_file)
else:
    # read circuit
    with open('algo_level_qc.qpy', 'rb') as qc_file:
        qc = qpy.load(qc_file)[0]
    with open('algo_level_qc_no_save_state.qpy', 'rb') as qc_file:
        new_qc = qpy.load(qc_file)[0]

# draw circuit
# new_qc.draw(output='mpl')

In [8]:
import trasyn
from qiskit import qpy
from qiskit.quantum_info import Statevector, state_fidelity

# optimize t-gate count with transyn
global syned_qc
if compute_on_the_fly:
    # perform gate synthesis
    syned_qc = trasyn.synthesize_qiskit_circuit(circuit=new_qc, u3_transpile=True, nonclifford_budget=50)
    
    # dump synthesized circuit to a file
    with open('trasyn_qc_reduced_t_gates.qpy', 'wb') as qc_file:
        qpy.dump(syned_qc, qc_file)
else:
    with open('trasyn_qc_reduced_t_gates.qpy', 'rb') as qc_file:
        syned_qc = qpy.load(qc_file)[0]

print("gate counts: ", dict(syned_qc.count_ops()))

# compute error after synthesis
original_state = Statevector(new_qc)
syned_state = Statevector(syned_qc)
fidelity = state_fidelity(state1=original_state, state2=syned_state)
print(f"state fidelity after synthesis: {fidelity}")

# draw cicuit
# syned_qc.draw(output='mpl')

gate counts:  {'h': 1540, 't': 1515, 'x': 280, 'y': 266, 'cx': 90, 's': 51, 'z': 2}
state fidelity after synthesis: 0.9998988393058632


In [12]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import OptimizeCliffords, CommutativeCancellation, RemoveResetInZeroState

# Step 1: Define safe passes
safe_passes = [
    RemoveResetInZeroState(),  # optional cleanup
    OptimizeCliffords(),       # optimize Clifford gates
    CommutativeCancellation(), # commute and cancel gates
]

# Step 2: Create a PassManager
pm = PassManager(safe_passes)

# Step 3: Run the passes manually
opt_qc = pm.run(syned_qc)  # <-- NOT transpile()

# Step 4 (optional): Transpile to ensure basis gates only
from qiskit import transpile
opt_qc = transpile(opt_qc, basis_gates=['h', 'x', 'y', 'z', 's', 'cx', 't'], optimization_level=2)

# Step 5: Done
print(dict(opt_qc.count_ops()))
opt_qc.draw('mpl')


TranspilerError: 'Unable to translate the operations in the circuit: ["u", "cx"] to the backend\'s (or manually specified) target basis: {"t", "reset", "measure", "y", "if_else", "z", "for_loop", "cx", "while_loop", "box", "s", "switch_case", "delay", "x", "h", "barrier", "snapshot", "store"}. This likely means the target basis is not universal or there are additional equivalence rules needed in the EquivalenceLibrary being used. For more details on this error see: https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes. BasisTranslator#translation-errors'